<a href="https://colab.research.google.com/github/decoz/mlclass/blob/master/ch7_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mask RCNN 

RCNN중에 가장 발전된 형태인 Mask-RCNN은 네트워크의 구조가 복잡해서 아직 케라스 내부 라이브러리에서 지원하지 안는다. 또한 동시에 엄청난 학습 시간을 필요로 하니 이번 시간엔 이를 구현한 외부코드를 이용해 사용하는 방법을 한번 체험해본다. 

먼저 가장 널리 알려진 Keras RCNN 코드는 https://github.com/matterport/Mask_RCNN.git 에 존재한다. 이 소스를 카피하는 git 명령어를 실행시켜보자 



### 설치 (matterport 버젼)

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

이 코드를 실행하기 위해서 필요한 패키지는 다음과 같다. 특히 텐서플로우의 하위버젼이 필요한데 이럴땐 colab 의 가상 환경이 도움이 되니 부담없이 다운그래이드를 해준다. 

In [ ]:
!pip install pycocotools
!pip install mrcnn|
!pip install tensorflow==1.9.0
!pip install keras==2.2.0
print("Installed packages")

작업을 원할하게 하기 위해 현재 디렉토리를 위에서 다운받은 폴더로 이동해준다. 

In [ ]:
import os
os.chdir('/content/Mask_RCNN/')

파이썬 소스에 setup.cfg 와 setup.py 가 있을 경우는 대부분 이 코드를 인스톨이 가능하다. 이를 인스톨 하는 명령은 다음과 같다. 

In [ ]:
!python setup.py install 

### 모델 로딩 

다음은 모델을 로딩하기 위한 준비단계이다.  먼저 모델을 파일로 다운로드 받도록 하겠다. 

In [ ]:

from mrcnn import utils
utils.download_trained_weights("mask_rcnn_coco.h5")


이제 좌측 파일 탭을 보면 Mask_RCNN 폴더에 mask_rcnn_coco.h5 파일이 생긴 것을 볼 수 있다.

mrcnn 은 모델을 생성시 config 가 필요한데  coco 데이터에 맞게 model 생성하기 위해 coco 모듈을 import 하고 이를 이용해 config 를 상속받아 설정한다. 

In [ ]:
import sys 
sys.path.append("/content/Mask_RCNN/samples/coco/") 

import coco
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

 간단하게 load_model 하면 되는 다른 모델과 달리 mrcnn 은 망구조와 weight 를 따로 로딩한다. config 가 생성되었으니 이제 이 config를 통해 모델을 생성하고 weight를 읽어들인다.  

In [ ]:
import mrcnn.model as modellib
model = modellib.MaskRCNN(mode="inference", config=config, model_dir="/content/Mask_RCNN/")
model.load_weights("mask_rcnn_coco.h5", by_name=True)


### 모델 테스트 

이제 모델로 이미지를 테스트 하기전에 각 클래스별 레이블이 필요하다. 이는 학습한 데이터에 따라 다른데 우리가 로딩한 데이터의 레이블은 다음과 같다. 


In [ ]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

일반적인 케라스 모델과는 달리 커스텀 모델을 정의해 쓰기 때문에 predict 대신 detect 를 사용한다. 그러면 먼저 이미지 디렉토리에 있는 샘플 이미지중에 랜덤 이미지를 읽어서 시각화 하는 코드를 보도록 하겠다. 



In [ ]:
from mrcnn import visualize
import os 
import skimage
import random 

IMAGE_DIR = "/content/Mask_RCNN//images"
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])

result 값 역시 일반적인 모델과 달리 dictionary 형태로 리턴되며 시각화 역시 mrcnn 에서 자체적으로 지원된다. 

### <font color = 'red'> 연습문제 7-1 : 직접 이미지를 다운 받아 테스트하기 
위의 클래스에 해당하는 사진이 있는 이미지를 받아 직접 업로드해서 테스트해보자. 
</font>